In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from simple_radio_script_encapsulated import _get_teaser_for_story
from simple_radio_script_encapsulated import teaser_matches_crit

In [3]:
story_no = 23188

In [4]:
t1 = _get_teaser_for_story(story_no)

In [5]:
crit_match_prompt = """
    You are a professional journalist in the medicine domain. You will receive a text and i want you to decide whether the following criteria matches the text.
    Please answer in the following format.
    The Criteria is: The Text mentions one or more persons
    Decision: yes or no. Reference: Paragraph in the text which matches the criteria.

    ```
    TEXT: {TEXT}
    ```
    Please answer in German.
    """ 

In [6]:
teaser_matches_crit(t1, crit_match_prompt)

'Die Kriterien sind: Der Text erwähnt eine oder mehrere Personen.\nEntscheidung: Ja. Referenz: Absatz 2 - "In einer großen Studie im Fachblatt „Nature Medicine“ hat nun ein internationales Forschungsteam fast eine Million Menschen in neun europäischen Ländern untersucht, die vor ihrem 22. Lebensjahr CT-Scans hatten (siehe Primärquelle)."'